# <center>Advanced MPI</center>

## Blocking and Non-blocking Communication in MPI

### Blocking Communication

Blocking communication operations in MPI, such as `MPI_Send` and `MPI_Recv`, are operations that do not return control to the user program until they are completed. This means that when a process calls `MPI_Send`, it will wait until the data has been copied out of the send buffer and is safe to be modified or reused. Similarly, when `MPI_Recv` is called, the process will wait until the data has been fully received and placed into the receive buffer.

Blocking operations are straightforward to use and are suitable for many applications. However, they can lead to inefficiencies in some scenarios where processes need to wait for each other, resulting in idle time.

![Blocking Communication](./img/blocking-comm.png)

**Example: Blocking Send and Receive**

This example demonstrates a simple blocking send and receive operation where one process sends a message to another.

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    if (world_rank == 0) {
        int data = 100;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent data %d to process 1\n", data);
    } else if (world_rank == 1) {
        int data;
        MPI_Recv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received data %d from process 0\n", data);
    }

    MPI_Finalize();
    return 0;
}
```

In [ ]:
!srun mpicc code/blocking_send_recv.c -o code/blocking_send_recv.o

In [ ]:
#program using 2,3,4 processes per node and using 2 and 3 nodes:
!srun -N 2 ./code/blocking_send_recv.o

### Non-blocking Communication

Non-blocking communication operations in MPI, such as `MPI_Isend` and `MPI_Irecv`, allow a process to initiate a communication operation and then proceed with other computations or communications without waiting for the communication to complete. These functions return immediately, providing a `MPI_Request` object that can be used to check the status or wait for the operation to complete.

Non-blocking operations are useful for overlapping communication with computation, potentially improving the performance of parallel applications by reducing idle time.
![Non Blocking Communication](./img/non-blocking-comm.png)


**Example: Non-blocking Send and Receive**

This example demonstrates non-blocking send and receive operations where one process sends a message to another, but both processes can perform other work while waiting for the communication to complete.

```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    int data;
    MPI_Request request;
    MPI_Status status;

    if (world_rank == 0) {
        data = 123;
        MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        printf("Process 0 initiated non-blocking send of data %d\n", data);
        // Perform some work while the send operation completes
        printf("Process 0 is doing other work while waiting for send to complete\n");
        MPI_Wait(&request, &status);  // Ensure the send operation is complete
    } else if (world_rank == 1) {
        MPI_Irecv(&data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);
        printf("Process 1 initiated non-blocking receive\n");
        // Perform some work while the receive operation completes
        printf("Process 1 is doing other work while waiting for receive to complete\n");
        MPI_Wait(&request, &status);  // Ensure the receive operation is complete
        printf("Process 1 received data %d\n", data);
    }

    MPI_Finalize();
    return 0;
}```

In [ ]:
!srun mpicc code/unit_1.c -o code/unit_1.o

In [ ]:
# Exercise No 4
# Save the program as `non_blocking_send_recv.c` on code folder code and compile it using `mpicc`:

!mpicc ...

#program using 2,3,4 processes per node and using 2 and 3 nodes:

!mpirun ...


### Comparison of Blocking and Non-blocking Communication

1. **Blocking Communication**:
   - **Pros**: Simplicity, straightforward usage.
   - **Cons**: Potential for idle time, processes may wait for each other.

2. **Non-blocking Communication**:
   - **Pros**: Overlap communication with computation, potential performance improvements.
   - **Cons**: More complex to use, requires careful management of `MPI_Request` objects and completion checks.

Using non-blocking communication effectively requires a good understanding of the application’s communication and computation patterns, enabling the overlap of these operations to maximize performance.

In [ ]:
!mpirun -np 4 ./hello_world # This example uses 4 processes on the same machine 
!!srun -N 2 -n 4 --ntasks-per-node=2 ./hello_world  # This example uses 4 processes on 2 nodes

Note that the execution block is enclosed in a function called `main()`, which returns the value 0 if it is completed successfully. The declaration of `main()` is mandatory in C/C++.

In [ ]:
!mpicc code/unit_1.c -o code/unit_1.o

In [ ]:
!srun -N 2 -n 4 --ntasks-per-node=2 ./code/unit_1.o

## MPI synchronization 
Refers to mechanisms that ensure coordination and consistency among multiple processes in a parallel computing environment. It helps control the order of execution, ensuring that processes operate in sync when necessary. Two key synchronization operations are:

-**MPI_Barrier:** This is a collective operation where all processes in a communicator must reach the barrier before any of them can proceed. It ensures that all processes are synchronized at a specific point in the program.

-**MPI_Wait:** This is used in conjunction with non-blocking communication, ensuring that a specific communication operation (e.g., MPI_Isend, MPI_Irecv) is completed before the process continues execution. It synchronizes a process with respect to the completion of communication tasks.

These synchronization primitives help manage timing and execution in distributed environments, avoiding race conditions and ensuring correct data handling

### Exercise: MPI Barrier and Wait with Non-blocking Communication
Objective: In this exercise, you will learn how to use MPI_Barrier for synchronizing all processes and MPI_Wait to handle non-blocking communication. You will modify the given code to ensure all processes reach a common point before proceeding, and ensure communication is completed before continuing execution.

#### Task:
- Initialize MPI and obtain the rank of the process.
- Use MPI_Isend and MPI_Irecv for non-blocking communication between two processes.
- Use MPI_Wait to ensure the communication is completed.
- Use MPI_Barrier to synchronize all processes after the communication is done.
- Ensure that process 1 receives the message before all processes print their rank.

#### Steps:
Set up MPI.
- Use MPI_Isend to send data asynchronously.
- Use MPI_Irecv to receive data asynchronously.
- Use MPI_Wait to ensure that communication is completed before proceeding.
- Use MPI_Barrier to ensure all processes are synchronized before printing their rank.

In [ ]:
!srun mpicc Exercises/exercise2.c -o Exercises/exercise2.o

In [ ]:
!srun -N 2 -n 4 --ntasks-per-node=2 ./exercises/exercise2.o